## Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

In [3]:
# define all dataframes for later
# gross_df = pd.read_csv('data/zipped_files/bom.movie_gross.csv.gz')
# movies_df = pd.read_csv('data/zipped_files/tmdb.movies.csv.gz')
budgets_df = pd.read_csv('data/zipped_files/tn.movie_budgets.csv.gz')
# tsv files have to be read with \t (tab spcaes) as their delimiter
# info_df = pd.read_csv('data/zipped_files/rt.movie_info.tsv.gz',delimiter='\t')
# file has an issue with encoding, read as cp850 or it throws an error
# reviews_df = pd.read_csv('data/zipped_files/rt.reviews.tsv.gz', delimiter='\t', encoding='cp850')

![database schema flow chart](./images/db_schema.jpeg)

In [4]:
#################################
# REQUIRES UNZIPPING data/zipped_files/im.db.zip
# INTO DIRECOTRY data/unzipped AS im.db
#################################
conn = sqlite3.connect('data/unzipped/im.db')
# call this later => imdb_df = pd.read_sql('''<QUERY>''',conn)

In [5]:
### GETTING NAMES OF ALL TABLE NAMES
tables = pd.read_sql(
"""
SELECT * FROM sqlite_master
""", conn)

In [6]:
imdb_directors = pd.read_sql("""
SELECT d.movie_id, p.primary_name FROM directors as d
LEFT JOIN persons AS p
    ON d.person_id = p.person_id
GROUP BY d.movie_id
""", conn)

In [7]:
imdb_all_movie_names = pd.read_sql("""
SELECT DISTINCT l.movie_id, l.title, r.primary_title, r.original_title
FROM movie_akas AS l
INNER JOIN movie_basics AS r
    ON l.movie_id = r.movie_id
""",conn)

In [14]:
pd.read_sql("""
SELECT * FROM movie_akas
""",conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [44]:
movie_akas = pd.read_sql("""
SELECT DISTINCT movie_id, title FROM movie_akas
""",conn)

In [111]:
movie_akas['title']

0                                   Джурасик свят
1                               Jurashikku warudo
2         Jurassic World: O Mundo dos Dinossauros
3                         O Mundo dos Dinossauros
4                                  Jurassic World
                           ...                   
266864                              Большая Акула
266865                         Sayonara kuchibiru
266866                              Farewell Song
266867                                La atención
266868                              The Attention
Name: title, Length: 266869, dtype: object

In [112]:
"The Attention" in list(movie_akas['title'])

True

In [118]:
movie_akas_list = list(movie_akas['title'])
movie_ids_list = list(movie_akas['movie_id'])

In [120]:
budgets_df['movie_id'] = [movie_ids_list[movie_akas_list.index( title )] if title in movie_akas_list else None for title in budgets_df['movie']]

In [126]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
 6   fixed_titles       0 non-null      object
 7   movie_id           2502 non-null   object
dtypes: int64(1), object(7)
memory usage: 361.5+ KB


In [127]:
len(budgets_df['movie_id'].unique())

2407

In [77]:
akas2 = movie_akas.groupby('movie_id').agg(lambda x: list(x))
akas2.head()

,title
movie_id,
tt0063540,"[Sangharsh, Sunghursh, Sungharsh]"
tt0066787,"[One Day Before the Rainy Season, Ashad Ka Ek ..."
tt0069049,"[La otra cara del viento, Druga strona wiatru,..."
tt0069204,"[Subse Bada Sukh, The Greatest Happiness, Sabs..."
tt0100275,"[La Telenovela Errante, The Wandering Soap Ope..."


In [90]:
# id_list = akas2.index[[akas2['title'].map(lambda x: title in x) for title in budgets_df['movie']]]


KeyboardInterrupt: 

In [ ]:
#akas2.index[akas2['title'] == ]# "Jurassic World: O Mundo dos Dinossauros"]

In [53]:
movie_akas[['movie_id','title']]

,movie_id,title
0,tt0369610,Джурасик свят
1,tt0369610,Jurashikku warudo
2,tt0369610,Jurassic World: O Mundo dos Dinossauros
3,tt0369610,O Mundo dos Dinossauros
4,tt0369610,Jurassic World
...,...,...
266864,tt9755806,Большая Акула
266865,tt9827784,Sayonara kuchibiru
266866,tt9827784,Farewell Song
266867,tt9880178,La atención


In [42]:
### ex ### fixed_titles.loc[['Frozen' in title for title in fixed_titles['title']]]
#[movie in title for title in movie_akas['titles'] for movie in budgets_df['movie']]
budgets_df['fixed_titles'] = budgets_df['movie'].map(akas_dict)
#movie_akas['movie_id'].loc[movie_akas[movie_akas['title'] == budgets_df['movie']]] # .loc[[title in titles for titles in movie_akas['titles'] for title in budgets_df['movie']]]
#for title in budgets_df['movie']:
#    for titles in movie_akas['titles']:
#        print(title in titles)

In [43]:
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,fixed_titles
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",NaN
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",NaN
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",NaN
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",NaN
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",NaN
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,NaN
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",NaN
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",NaN
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,NaN


In [356]:
recent_imdb_movies = pd.read_sql("""
SELECT * FROM movie_basics
WHERE CAST(start_year AS int) BETWEEN 2013 AND 2023
""", conn)

In [357]:
recent_imdb_movies.sort_values('start_year')

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
17314,tt2971400,Loving Memory,Loving Memory,2013,NaN,Drama
17313,tt2971388,308,308,2013,121.0,"Horror,Mystery,Thriller"
17312,tt2971128,"River Deep, Mountain High: James Nesbitt in Ne...","River Deep, Mountain High: James Nesbitt in Ne...",2013,NaN,Documentary
17311,tt2970792,Wechselspiel,Wechselspiel,2013,73.0,Drama
...,...,...,...,...,...,...
5999,tt1757678,Avatar 3,Avatar 3,2023,NaN,"Action,Adventure,Drama"
74209,tt6495056,Untitled Illumination Entertainment Project,Untitled Illumination Entertainment Project,2023,NaN,None
2392,tt10298848,Untitled Disney Live-Action Project,Untitled Disney Live-Action Project,2023,NaN,None
70264,tt6258542,Wraith of the Umbra and Eidolon II,Wraith of the Umbra and Eidolon II,2023,NaN,"Adventure,Drama,Fantasy"


In [359]:
recent_imdb_movies.value_counts('start_year')

start_year
2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2019     8379
2020      937
2021       83
2022       32
2023        5
dtype: int64

In [341]:
fixed_titles.loc[['Frozen' in title for title in fixed_titles['title']]]

,title,studio,domestic_gross,foreign_gross,year,movie_id,primary_title,original_title
0,Frozen,BV,400700000.0,875700000,2013,tt1323045,Frozen,Frozen
1,Frozen,BV,400700000.0,875700000,2013,tt2294629,Frozen,Frozen
2,Frozen,BV,400700000.0,875700000,2013,tt1611845,Frozen,Wai nei chung ching


In [392]:
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495,192013
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,0
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,0


In [360]:
# fixed_titles = budgets_df.merge(aka_movies, how='left',  left_on='movie', right_on='title')

In [362]:
fixed_titles.isna().sum()

id                      0
release_date            0
movie                   0
production_budget       0
domestic_gross          0
worldwide_gross         0
movie_id             3280
title                3280
primary_title        3280
original_title       3280
dtype: int64

In [386]:
fixed_titles.dropna(subset=['movie_id'],inplace = True)

In [377]:
# convert dollar amounts (str) into int
budgets_df['domestic_gross'] = budgets_df['domestic_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['worldwide_gross'] = budgets_df['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['production_budget'] = budgets_df['production_budget'].str.replace('$', '').str.replace(',', '').astype(int)
# calculate foreign gross based off worldwide - domestic values
budgets_df['foreign_gross'] = budgets_df['worldwide_gross'] - budgets_df['domestic_gross']

AttributeError: Can only use .str accessor with string values!

In [376]:
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365


In [379]:
fixed_titles.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,movie_id,title,primary_title,original_title
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",tt1775309,Avatar,Avatar,Abatâ
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",tt1298650,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",tt6565702,Dark Phoenix,Dark Phoenix,Dark Phoenix
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",tt2395427,Avengers: Age of Ultron,Avengers: Age of Ultron,Avengers: Age of Ultron
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",NaN,NaN,NaN,NaN


In [382]:
movies_with_gross_df = recent_imdb_movies.merge(recent_gross_df,left_on='primary_title',right_on='title',how='inner')

In [385]:
fixed_titles.dropna(subset=['movie_id']).sort_values('domestic_gross',ascending=False)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,movie_id,title,primary_title,original_title
780,43,"Aug 3, 2018",Christopher Robin,"$75,000,000","$99,215,042","$197,504,758",tt4575576,Christopher Robin,Christopher Robin,Christopher Robin
868,8,"Jun 13, 1997",Hercules,"$70,000,000","$99,112,101","$250,700,000",tt3499424,Hercules,Hercules Reborn,Hercules Reborn
869,8,"Jun 13, 1997",Hercules,"$70,000,000","$99,112,101","$250,700,000",tt3985956,Hercules,Hercules,Hercules
867,8,"Jun 13, 1997",Hercules,"$70,000,000","$99,112,101","$250,700,000",tt1267297,Hercules,Hercules,Hercules
870,9,"Mar 22, 2013",Olympus Has Fallen,"$70,000,000","$98,927,592","$172,878,928",tt2302755,Olympus Has Fallen,Olympus Has Fallen,Olympus Has Fallen
...,...,...,...,...,...,...,...,...,...,...
6101,69,"Sep 18, 1967",Point Blank,"$3,000,000",$0,$0,tt7381042,Point Blank,Point Blank,Point Blank
6104,72,"Aug 14, 2015",Amnesiac,"$3,000,000",$0,$0,tt2837336,Amnesiac,Amnesiac,Amnesiac
6105,72,"Aug 14, 2015",Amnesiac,"$3,000,000",$0,$0,tt2693114,Amnesiac,Amnesiac,Amnesiac
6106,73,"Dec 31, 2015",Unnatural,"$3,000,000",$0,$0,tt4373974,Unnatural,Beyond the Bridge,Beyond the Bridge


In [324]:
# movie_akas.groupby(by='movie_id').agg(lambda x: list(x))